In [1]:
from pyscf import scf,gto
import numpy as np
import inspect
from FcMole import FcM
from functools import reduce
import matplotlib.pyplot as plt
from pyscf.scf import cphf
from pyscf.grad import rhf as grhf  #### very important
from pyscf.hessian import rhf as hrhf # without those two mf.Gradients() and mf.Hessian() don't work
def DeltaV(mol,dL):
    mol.set_rinv_orig_(mol.atom_coords()[0])
    dV=mol.intor('int1e_rinv')*dL[0]
    mol.set_rinv_orig_(mol.atom_coords()[1])
    dV+=mol.intor('int1e_rinv')*dL[1]
    return -dV

In [2]:
mol=gto.M(atom="H 0 0 0; Li 0 0 1.8",unit="Bohr",basis="STO-3G")
mf=scf.RHF(mol)
mf.scf(dm0=mf.init_guess_by_atom())
P=mf.make_rdm1()
hcore=mf.get_hcore()
h=mf.Hessian()
h.kernel()
pass

The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.
converged SCF energy = -7.74106569060784


/home/giorgiod/miniconda3/lib/python3.7/site-packages/pyscf/lib/chkfile.py:71: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  return val.value


In [3]:
mol1=gto.M(atom="H 0 0 0; Li 0 0 1.801",unit="Bohr",basis="STO-3G")
mf1=scf.RHF(mol1)
mf1.scf()
hcore1=mf1.get_hcore()
P1=mf1.make_rdm1()
h1=mf1.Hessian()
h1.kernel()
pass

converged SCF energy = -7.7413864568394


In [4]:
dP_fd=P1-P
print(dP_fd) #note px,py orbitals of Li are not populatedv

[[-1.06937710e-05  3.16008435e-05  5.58477228e-05  0.00000000e+00
   0.00000000e+00  8.98721281e-05]
 [ 3.16008435e-05 -7.80643728e-05  1.61159419e-04  0.00000000e+00
   0.00000000e+00 -1.93316647e-04]
 [ 5.58477228e-05  1.61159419e-04  7.79286976e-05  0.00000000e+00
   0.00000000e+00  2.88893842e-05]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 8.98721281e-05 -1.93316647e-04  2.88893842e-05  0.00000000e+00
   0.00000000e+00 -1.34563955e-04]]


In [5]:
mo_energy = mf.mo_energy
mo_occ = mf.mo_occ
mo_coeff = mf.mo_coeff
C=mo_coeff
atmlst = range(mol.natm)
mocc = mo_coeff[:,mo_occ>0]

In [6]:
h1ao = h.make_h1(mo_coeff, mo_occ,)

In [7]:
h1ao[1].shape

(3, 6, 6)

In [8]:
mo1, mo_e1=h.solve_mo1(mo_energy,mo_coeff,mo_occ,h1ao)
mo1_a0,mo1_a1=mo1[0],mo1[1]

In [9]:
dm1_0 = np.einsum('ypi,qi->ypq', mo1_a0, mocc) #shape=(nocc,nocc)

In [10]:
print(dm1_0[2])

[[ 0.00263533  0.01456258  0.00158421  0.          0.         -0.00106684]
 [-0.03033904  0.01951954 -0.02133932  0.          0.          0.02166091]
 [-0.02949064 -0.05931378 -0.01944546  0.          0.          0.01713226]
 [ 0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.        ]
 [-0.04388821  0.07506171 -0.03164511  0.          0.          0.03368086]]


In [11]:
mo1_a1[2]

array([[-0.01576236, -0.00349589],
       [-0.00456256,  0.05589252],
       [ 0.07337374,  0.04778517],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [-0.05313872,  0.08476794]])

In [12]:
U=np.zeros((6,6))
U[:2,:2]=mo1_a1[2]
O=np.diag(np.array([1,1,1,0,0,0]))

ValueError: could not broadcast input array from shape (6,2) into shape (2,2)

The formula for the Hessian is a sum of 6 part, the first 3 are computed by ess_elec function:
$$ \frac{\partial E}{\partial x \partial y}= \sum_{\mu\nu} \frac{\partial P_{\mu\nu}}{\partial y}\frac{\partial H_{\mu\nu}}{\partial x}-\frac{\partial W_{\mu\nu}}{\partial y} \frac{\partial S_{\mu\nu}}{\partial x}
+ \sum_{\mu\nu\lambda\sigma} \frac{\partial P_{\mu\nu}}{\partial y} P_{\lambda \sigma}(\frac{\partial}{\partial x})(\mu \lambda || \nu \sigma) + ...
$$ The remaining are computed by partial_hess_elec function

In [ ]:
"""
The function for electronic hessian 
ess_elec(hessobj, mo_energy=None, mo_coeff=None, mo_occ=None,mo1=None, mo_e1=None, h1ao=None,atmlst=None, max_memory=4000, verbose=None):

    mol = hessobj.mol
    mf = hessobj.base
    if mo_energy is None: mo_energy = mf.mo_energy
    if mo_occ is None:    mo_occ = mf.mo_occ
    if mo_coeff is None:  mo_coeff = mf.mo_coeff
    if atmlst is None: atmlst = range(mol.natm)
    de2 = hessobj.partial_hess_elec(mo_energy, mo_coeff, mo_occ, atmlst, max_memory, log)
    if h1ao is None:
        h1ao = hessobj.make_h1(mo_coeff, mo_occ, hessobj.chkfile, atmlst, log)           #dH/dx + dVee/dx
    if mo1 is None or mo_e1 is None:
        mo1, mo_e1 = hessobj.solve_mo1(mo_energy, mo_coeff, mo_occ, h1ao, None, atmlst, max_memory, log)

    nao, nmo = mo_coeff.shape
    mocc = mo_coeff[:,mo_occ>0]
    s1a = -mol.intor('int1e_ipovlp', comp=3)      ######   dS/dx   same as gradients

    aoslices = mol.aoslice_by_atom()
    for i0, ia in enumerate(atmlst):    #for every atom i 
        shl0, shl1, p0, p1 = aoslices[ia]
        s1ao = numpy.zeros((3,nao,nao))
        s1ao[:,p0:p1] += s1a[:,p0:p1]
        s1ao[:,:,p0:p1] += s1a[:,p0:p1].transpose(0,2,1)
        s1oo = numpy.einsum('xpq,pi,qj->xij', s1ao, mocc, mocc)

        for j0 in range(i0+1):   #for every atom j != i   
            ja = atmlst[j0]
            q0, q1 = aoslices[ja][2:]
# *2 for double occupancy, *2 for +c.c.
            dm1 = numpy.einsum('ypi,qi->ypq', mo1[ja], mocc)                          #   dP/dy
            de2[i0,j0] += numpy.einsum('xpq,ypq->xy', h1ao[ia], dm1) * 4              #   dP/dy*(dH/dx + d(||)/dx)
            dm1 = numpy.einsum('ypi,qi,i->ypq', mo1[ja], mocc, mo_energy[mo_occ>0])   #   dW/dy 
            de2[i0,j0] -= numpy.einsum('xpq,ypq->xy', s1ao, dm1) * 4                  #  -dW/dy * dS/dx
            de2[i0,j0] -= numpy.einsum('xpq,ypq->xy', s1oo, mo_e1[ja]) * 2

        for j0 in range(i0):
            de2[j0,i0] = de2[i0,j0].T 
    return de2
"""
pass

In [ ]:
aoslices = mol.aoslice_by_atom()
for i0, ia in enumerate(range(0, 4)):
    print(i0, ia)
atmlst

In [ ]:
h1ao=h.make_h1(mo_coeff,mo_occ)

In [ ]:
mo1, mo_e1=h.solve_mo1(mo_energy, mo_coeff,mo_occ,h1ao)

In [ ]:
mo_e1[0].shape,mo1[0].shape

In [ ]:
dm1 = np.einsum('ypi,qi->ypq', mo1[1], mocc)
print(np.round(dm1[2]*4,decimals=4))  
print(np.round((dm1[2]+(dm1[2]).T)*2,decimals=4))  
print(np.round(dP_fd*1000,decimals=4))
print(np.round(dm1[2]*4,decimals=4).diagonal())  #troppo uguali per essere un caso
print(np.round(dP_fd*1000,decimals=4).diagonal())

In [ ]:
np.einsum('xpq,ypq->xy', h1ao[1], dm1) * 4 #component of the hessian

In [ ]:
"""
def solve_mo1(mf, mo_energy, mo_coeff, mo_occ, h1ao_or_chkfile,
              fx=None, atmlst=None, max_memory=4000, verbose=None):
    '''Solve the first order equation

    Kwargs:
        fx : function(dm_mo) => v1_mo
            A function to generate the induced potential.
            See also the function gen_vind.
    '''
    mol = mf.mol
    if atmlst is None: atmlst = range(mol.natm)

    nao, nmo = mo_coeff.shape
    mocc = mo_coeff[:,mo_occ>0]
    nocc = mocc.shape[1]

    if fx is None:
        fx = gen_vind(mf, mo_coeff, mo_occ)
    s1a = -mol.intor('int1e_ipovlp', comp=3)

    def _ao2mo(mat):
        return numpy.asarray([reduce(numpy.dot, (mo_coeff.T, x, mocc)) for x in mat])

    mo1s = [None] * mol.natm
    e1s = [None] * mol.natm
    aoslices = mol.aoslice_by_atom()
    for ia0, ia1 in lib.prange(0, len(atmlst), blksize):
        s1vo = []
        h1vo = []
        for i0 in range(ia0, ia1):
            ia = atmlst[i0]
            shl0, shl1, p0, p1 = aoslices[ia]
            s1ao = numpy.zeros((3,nao,nao))
            s1ao[:,p0:p1] += s1a[:,p0:p1]
            s1ao[:,:,p0:p1] += s1a[:,p0:p1].transpose(0,2,1)
            s1vo.append(_ao2mo(s1ao))
            if isinstance(h1ao_or_chkfile, str):
                key = 'scf_f1ao/%d' % ia
                h1ao = lib.chkfile.load(h1ao_or_chkfile, key)
            else:
                h1ao = h1ao_or_chkfile[ia]
            h1vo.append(_ao2mo(h1ao))

        h1vo = numpy.vstack(h1vo)
        s1vo = numpy.vstack(s1vo)
        mo1, e1 = cphf.solve(fx, mo_energy, mo_occ, h1vo, s1vo)
        mo1 = numpy.einsum('pq,xqi->xpi', mo_coeff, mo1).reshape(-1,3,nao,nocc)    ###### imortant C@U
        e1 = e1.reshape(-1,3,nocc,nocc) 

        for k in range(ia1-ia0):
            ia = atmlst[k+ia0]
            if isinstance(h1ao_or_chkfile, str):
                key = 'scf_mo1/%d' % ia
                lib.chkfile.save(h1ao_or_chkfile, key, mo1[k])
            else:
                mo1s[ia] = mo1[k]
            e1s[ia] = e1[k].reshape(3,nocc,nocc)
        mo1 = e1 = None

    if isinstance(h1ao_or_chkfile, str):
        return h1ao_or_chkfile, e1s
    else:
        return mo1s, e1s
"""
pass

In [ ]:
fx = hrhf.gen_vind(mf, mo_coeff, mo_occ)

In [ ]:
h1ao=h.make_h1(mo_coeff,mo_occ)
print(hrhf.solve_mo1(mf, mo_energy, mo_coeff, mo_occ, h1ao)[0][0][2])
print(hrhf.solve_mo1(mf, mo_energy, mo_coeff, mo_occ, h1ao)[0][1][2])

In [ ]:
from pyscf import lib
import numpy
h1ao_or_chkfile=h.make_h1(mo_coeff,mo_occ)
nao, nmo = mo_coeff.shape
mocc = mo_coeff[:,mo_occ>0]
nocc = mocc.shape[1]

fx = hrhf.gen_vind(mf, mo_coeff, mo_occ)
s1a = -mol.intor('int1e_ipovlp', comp=3)
blksize = max(2, int(4000*1e6/8 / (nmo*nocc*3*6)))
def _ao2mo(mat):
    return numpy.asarray([reduce(numpy.dot, (mo_coeff.T, x, mocc)) for x in mat])

mo1s = [None] * mol.natm
e1s = [None] * mol.natm
aoslices = mol.aoslice_by_atom()
for ia0, ia1 in lib.prange(0, len(atmlst), blksize):
    s1vo = []
    h1vo = []
    for i0 in range(ia0, ia1):
        ia = atmlst[i0]
        shl0, shl1, p0, p1 = aoslices[ia]
        s1ao = numpy.zeros((3,nao,nao))
        s1ao[:,p0:p1] += s1a[:,p0:p1]
        s1ao[:,:,p0:p1] += s1a[:,p0:p1].transpose(0,2,1)
        s1vo.append(_ao2mo(s1ao))
        if isinstance(h1ao_or_chkfile, str):
            key = 'scf_f1ao/%d' % ia
            h1ao = lib.chkfile.load(h1ao_or_chkfile, key)
        else:
            h1ao = h1ao_or_chkfile[ia]
        h1vo.append(_ao2mo(h1ao))
    h1vo = numpy.vstack(h1vo)
    s1vo = numpy.vstack(s1vo)
    mo1, e1 = cphf.solve(fx, mo_energy, mo_occ, h1vo, s1vo)

In [ ]:
mo1[5]
mo1_2 = numpy.einsum('pq,xqi->xpi', mo_coeff, mo1).reshape(-1,3,nao,nocc)

In [ ]:
print(np.round(C,decimals=2),'\n'*2,mo1[5],'\n'*2,mo1_2[1][2],'\n'*2,C@mo1[5]@mocc.T)

In [ ]:
dm1[2]

In [ ]:
print(hrhf.solve_mo1(mf, mo_energy, mo_coeff, mo_occ, h1ao)[0][1][2])

In [ ]:
U=np.zeros_like(C)
U[:,:2]=mo1[1][2].copy()
U=U+U.T

In [ ]:
C@(U@O-O@U.T)@C.T

In [ ]:
np.round(dP_fd*1000,decimals=3)